# Diversity Statements Scraper

In [1]:
# Data manipulation libraries
import pandas as pd
import numpy as np
# Common webscraping libaries
from bs4 import BeautifulSoup as bs
import requests

### Get list of universities

In [2]:
input_path = "../data/input/pilot_websites.csv"
uni_list = pd.read_csv(input_path)
uni_list = uni_list.melt(id_vars=['University'], 
              value_vars=[x for x in uni_list.columns if "WebPage" in x or "Admissions" in x], 
              var_name='url_type', value_name='url')
uni_list["url_type"],uni_list["url_rank"] = uni_list["url_type"].str[:-3], uni_list["url_type"].str[-3:]
uni_list = uni_list[["University","url_type","url_rank","url"]].dropna(subset=["url"])
uni_list

,University,url_type,url_rank,url
0,Louisiana State University and Agricultural & ...,WebPageLink,1.1,https://www.lsu.edu/diversity/about_us/mission...
1,University of California-San Diego,WebPageLink,1.1,https://ucsd.edu/campus-life/diversity/index.html
2,Saint Louis University,WebPageLink,1.1,https://www.slu.edu/about/key-facts/diversity/...
3,SUNY College of Environmental Science and Fore...,WebPageLink,1.1,https://www.esf.edu/ide/
4,Stony Brook University,WebPageLink,1.1,https://www.stonybrook.edu/diversity/
5,George Washington University,WebPageLink,1.1,https://diversity.gwu.edu/vision-and-mission
6,University of New Mexico-Main Campus,WebPageLink,1.1,https://diverse.unm.edu/about/mission.html
7,Duke University,WebPageLink,1.1,https://oie.duke.edu/mission
8,University of La Verne,WebPageLink,1.1,https://laverne.edu/diversity/
9,Loyola University Chicago,WebPageLink,1.1,https://www.luc.edu/diversityandinclusion/abou...


In [4]:
from markdownify import markdownify
import time
import re

def scrape_webpage(row):
    uni,url,url_type = row["University"],row["url"],row["url_type"]
    print(uni,url)
    r = requests.get(url,headers = {'User-Agent': 'Mozilla/5.0'})
    soup = bs(r.text).find("body")
    # Get rid of all JS, style forms, footers, headers and navs
    for script in soup(["script", "style","footer","header","nav"]):
        script.decompose()    # rip it out
    # Get rid of all divs that have the id nav[something]
    for tag in soup.findAll("div", {"id" : re.compile('nav*')}):
        tag.decompose()
    # Get rid of all divs that have the id nav[something]
    for tag in soup.findAll("div", {"class" : re.compile('nav*')}):
        tag.decompose()
        
    for hidden in soup.findAll(style=['display:none',
                                      'display:hidden',
                                      'visibility:none',
                                      'visibility:hidden',
                                      'position:absolute !important',
                                      'position:absolute',
                                      'display: none',
                                      'display: hidden',
                                      'visibility: none',
                                      'visibility: hidden',
                                      'position: absolute !important',
                                      'position: absolute',
                                     ]):
        hidden.decompose()
        

    overview = markdownify(str(soup))
    bl = "\n** **\n\n"
    overview = f"{url_type} – {url} \n\n {overview} {bl}"
    with open(f"../data/output/{uni.strip()}.md", "a", encoding='utf-8') as file:
        file.write(str(overview))
    time.sleep(1)

uni_list.apply(scrape_webpage,axis=1)


Louisiana State University and Agricultural & Mechanical College https://www.lsu.edu/diversity/about_us/mission_vision.php
University of California-San Diego https://ucsd.edu/campus-life/diversity/index.html
Saint Louis University https://www.slu.edu/about/key-facts/diversity/index.php
SUNY College of Environmental Science and Forestry https://www.esf.edu/ide/
Stony Brook University https://www.stonybrook.edu/diversity/
George Washington University https://diversity.gwu.edu/vision-and-mission
University of New Mexico-Main Campus https://diverse.unm.edu/about/mission.html
Duke University  https://oie.duke.edu/mission
University of La Verne https://laverne.edu/diversity/
Loyola University Chicago  https://www.luc.edu/diversityandinclusion/about/diversitystatement/
University of California-Santa Barbara  https://diversity.ucsb.edu/
Clark University  https://www.clarku.edu/who-we-are/diversity-and-inclusion/
Widener University  https://www.widener.edu/student-experience/vibrant-community/d

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
23    None
78    None
79    None
80    None
81    None
82    None
83    None
84    None
85    None
86    None
87    None
88    None
89    None
90    None
dtype: object